##  Imoprt package
DATASETS # 需先創建DATASETS，檔案內容為下列
* audio-dataset
    * submission
        *task1_answer.txt #放task1做完的結果
    * aicup.py 
    * task1_answer.txt #訓練資料
    * task1_answer_timestamps.json #做task2才需要 
    * task2_answer.txt #訓練資料

In [1]:
!pip install  datasets evaluate jiwer librosa
!pip install --upgrade bitsandbytes transformers==4.50.0 accelerate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 65.1 MB/s eta 0:00:00:00:01
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cesium 0.12.4 requires numpy<3.0,>=2.0, but you have numpy 1.26.4 which is incompatible.
bigframes 1.42.0 requires rich<14,>=12.4.4, but you have rich 14.0.0 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.9.0.13 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cudnn-cu12==9.1.0.70; pla

In [2]:
import os
import sys
sys.path.append('/kaggle/input/audio-dataset')
import json
import torch
import random
import librosa
import zipfile
import evaluate
import numpy as np
import pandas as pd
import re
from datasets import Dataset, Audio, load_dataset, Features, Value
from aicup import (DataCollatorSpeechSeq2SeqWithPadding,
      transcribe_with_timestamps,
      collate_batch_with_prompt_template,
      generate_annotated_audio_transcribe_parallel,OpenDeidBatchSampler)
from transformers import (
    WhisperFeatureExtractor, WhisperTokenizer, WhisperProcessor,
    WhisperForConditionalGeneration, Seq2SeqTrainingArguments, Seq2SeqTrainer, AutoTokenizer
)

2025-06-08 03:01:35.181951: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1749351695.406811      35 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1749351695.468514      35 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


# Task 2

In [3]:
def normalize_word(word):
    return re.sub(r'\s+', '', word.lower().strip().replace('.', ''))

# 載入原始 timestamps
with open("/kaggle/input/audio-dataset/task1_answer_timestamps.json", "r", encoding="utf-8") as f:
    timestamps = json.load(f)

# 加上 normalized 欄位
for file_id in timestamps:
    for segment in timestamps[file_id]["segments"]:
        if "word" in segment:
            segment["normalized"] = normalize_word(segment["word"])

# 選擇：要覆蓋原檔或寫入新檔
with open("/kaggle/working/task1_answer_timestamps.json", "w", encoding="utf-8") as f:  # 也可以改成其他檔名
    json.dump(timestamps, f, ensure_ascii=False, indent=4)

print("✅ 已加入 normalized 欄位到 task1 timestamps，可進行更穩定的比對")

✅ 已加入 normalized 欄位到 task1 timestamps，可進行更穩定的比對


In [4]:
task2_train_answer = r"/kaggle/input/audio-dataset/task2_answer.txt"
task2_train_transcribe = r"/kaggle/input/audio-dataset/task1_answer.txt"
task2_train_data = '/kaggle/working/task2_train.tsv'
generate_annotated_audio_transcribe_parallel(task2_train_answer, task2_train_transcribe, task2_train_data, num_processes=4)

process annotation file...
annotation file done
processing each medical file
1101	the things that you feel like nothing is going to come of this anyway? It is. And it's also feeling frustrating to spend like a couple of hours and all I can say is, "I looked through the job boards for this, this and this." Mm. "And there was nothing. And that time is gone and I still don't have a job." Mm. It's just, yeah, it's feeling daunting and exhausting. Yeah.	DURATION:a couple of hours

11658
All medical file done
write out to tsv format...
tsv format dataset done


## Read dataset

In [5]:
from datasets import load_dataset, Features, Value

task2_data = load_dataset("csv",data_files=task2_train_data, delimiter='\t',
  features = Features({'fid': Value('string'),'content': Value('string'),'label':Value('string')}),
  column_names=['fid','content','label'])

Generating train split: 0 examples [00:00, ? examples/s]

In [6]:
task2_data

DatasetDict({
    train: Dataset({
        features: ['fid', 'content', 'label'],
        num_rows: 11651
    })
})

In [7]:
from collections import Counter


ctr = Counter()

for i in task2_data['train']:
  phi_labelwvalue = i['label'].split("\\n")
  phi_label = [j.split(":")[0] for j in phi_labelwvalue]
  ctr.update(phi_label)

print(ctr)

Counter({'DATE': 1152, 'DOCTOR': 934, 'PHI': 864, 'ID_NUMBER': 835, 'ZIP': 742, 'COUNTRY': 723, 'LOCATION-OTHER': 666, 'DISTRICT': 642, 'PATIENT': 636, 'DURATION': 625, 'COUNTY': 602, 'SET': 500, 'TIME': 481, 'PERSONALNAME': 441, 'FAMILYNAME': 374, 'PHONE': 360, 'AGE': 336, 'EMAIL': 334, 'MEDICAL_RECORD_NUMBER': 329, 'ROOM': 323, 'PROFESSION': 322, 'ORGANIZATION': 321, 'USERNAME': 321, 'FAX': 318, 'CITY': 314, 'HOSPITAL': 313, 'STATE': 312, 'STREET': 308, 'DEPARTMENT': 307, 'URL': 284, 'IPADDRESS': 247})


## Model imoprt

In [8]:
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, AutoConfig
import torch
from peft import prepare_model_for_kbit_training, LoraConfig, get_peft_model, PeftModel # 確保導入 PeftModel

task2_model_name = "deepseek-ai/deepseek-llm-7b-base"

bos = '<|endoftext|>'
eos = '<|END|>'
pad = '<|pad|>'
sep = '\n\n####\n\n'

special_tokens_dict = {
    'eos_token': eos,
    'bos_token': bos,
    'pad_token': pad,
    'sep_token': sep
}

tokenizer = AutoTokenizer.from_pretrained(task2_model_name, use_fast=False, trust_remote_code=True)
tokenizer.padding_side = 'left'
num_added_toks = tokenizer.add_special_tokens(special_tokens_dict)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
)

config = AutoConfig.from_pretrained(
    task2_model_name,
    bos_token_id=tokenizer.bos_token_id,
    eos_token_id=tokenizer.eos_token_id,
    pad_token_id=tokenizer.pad_token_id,
    sep_token_id=tokenizer.sep_token_id, # 確認這裡使用 tokenizer.sep_token_id
    output_hidden_states=False
)

# **修改這裡：將模型載入基礎模型，準備 Kbit 訓練，但先不應用 LoRA**
# LoRA 會在後續的檢查點載入邏輯中應用
base_model = AutoModelForCausalLM.from_pretrained(
    task2_model_name,
    quantization_config=bnb_config,
    config=config,
    trust_remote_code=True
)
base_model.resize_token_embeddings(len(tokenizer))
base_model = prepare_model_for_kbit_training(base_model)

lora_config = LoraConfig(
    r=8,
    lora_alpha=32,
    target_modules=["q_proj", "v_proj","k_proj", "o_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

# 暫時不在這裡執行 get_peft_model，它會在後續的載入邏輯中執行
# model = get_peft_model(model, lora_config)

print(f"{tokenizer.pad_token}: {tokenizer.pad_token_id}")

tokenizer_config.json:   0%|          | 0.00/792 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/4.61M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/584 [00:00<?, ?B/s]

`low_cpu_mem_usage` was None, now default to True since model is quantized.


pytorch_model.bin.index.json:   0%|          | 0.00/22.5k [00:00<?, ?B/s]

...

pytorch_model-00001-of-00002.bin:   0%|          | 0.00/9.97G [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.6k [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/121 [00:00<?, ?B/s]

<|pad|>: 100017


In [9]:
def collate_batch_with_prompt_template(batch, tokenizer, template =
  "<|endoftext|> __CONTENT__\n\n####\n\n__LABEL__ <|END|>", IGNORED_PAD_IDX = -100):

  texts = [template.replace("__LABEL__", data['label']).replace("__CONTENT__",
    data['content']) for data in list(batch)]
  encoded_seq = tokenizer(texts, padding=True)

  indexed_tks = torch.tensor(encoded_seq['input_ids'])
  attention_mask = torch.tensor(encoded_seq['attention_mask'])
  encoded_label = torch.tensor(encoded_seq['input_ids'])
  encoded_label[encoded_label == tokenizer.pad_token_id] = IGNORED_PAD_IDX

  return indexed_tks, encoded_label, attention_mask

In [10]:
from torch.utils.data import Dataset, DataLoader


train_data = list(task2_data['train'])
train_dataloader = DataLoader(train_data, batch_size=2, shuffle=False, collate_fn=lambda batch: collate_batch_with_prompt_template(batch, tokenizer))
titer = iter(train_dataloader)
tks, labels, masks= next(titer)
print(tks.shape)
print(tks)
print(labels)
print(masks)
print(tokenizer.decode(tks[0]))
print(tokenizer.decode(tks[1]))
next(iter(titer))


torch.Size([2, 172])
tensor([[100017, 100017, 100017, 100017, 100017, 100017, 100017, 100017, 100017,
         100017, 100017, 100017, 100017, 100017, 100017, 100017, 100017, 100017,
         100017, 100017, 100017, 100017, 100017, 100017, 100017, 100017, 100017,
         100017, 100017, 100017, 100017, 100017, 100017, 100017, 100017, 100017,
         100017, 100017, 100017, 100017, 100017, 100017, 100017, 100017, 100017,
         100017, 100017, 100017, 100017, 100017, 100017, 100017, 100017, 100017,
         100017, 100017, 100017, 100017, 100017, 100017, 100017, 100017, 100017,
         100017, 100017, 100017, 100017, 100017, 100017, 100017, 100017, 100017,
         100017, 100017, 100017, 100017, 100017, 100017, 100017, 100017, 100017,
         100017, 100017, 100017, 100017, 100017, 100017, 100017, 100017, 100017,
         100017, 100017, 100017, 100017, 100017, 100017, 100017, 100017, 100017,
         100017, 100017, 100017, 100000, 100016,   6295,  60656,    453,    280,
       

(tensor([[100017, 100017, 100017, 100017, 100017, 100017, 100017, 100017, 100017,
          100017, 100017, 100017, 100017, 100017, 100017, 100017, 100017, 100017,
          100017, 100017, 100017, 100017, 100017, 100017, 100017, 100017, 100017,
          100017, 100017, 100017, 100017, 100017, 100017, 100017, 100017, 100017,
          100017, 100017, 100017, 100000, 100016,   2682,  20061,   3190,    245,
            2567,   3822,    280,    586,    441,   1435,    245,   8110,   3517,
           11054,     11,   1236,   2451,    362,   7213,   3517,    754,    362,
             438,   6524,   1555,   1712,    285,    643,    803,   9034,    359,
            1962,    937,    285,  19926,  13189,   2274,     13,    304,   1378,
             362,    953,   5131,    359,    372,    274,  20605,     11,    548,
             366,    254,   4473,   1734,     11,    340,    481,    298,   1189,
             359,    565,    340,   1113,      6,     83,   2579,    285,    745,
               6

In [11]:
results = tokenizer(
    [f"{bos} Yeah, I imagine it would — sorry, go ahead. So it's supposed to work immediately, right? Yep. So we'll see if I'm productive tomorrow. I hope I'm productive today. I've actually been trying to plan. If I do the titles today, then I can do my laundry tomorrow. Right. I probably could bring my computer and do titles while I'm doing my laundry. If I was — but I won't do that.{sep}DATE:tomorrow\nDATE:today\nDate:today {eos}",
    f"{bos} I imagine it{sep}PHI:Null {eos}"]
    ,padding=True
)
print(results['attention_mask'][0])
print(results['attention_mask'][1])
print(tokenizer.decode(results['input_ids'][0]))
print(tokenizer.decode(results['input_ids'][1]))

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
<｜begin▁of▁sentence｜><|endoftext|> Yeah, I imagine it would — sorry, go ahead. So it's supposed to work immediately, right? Yep. So we'll see if I'm productive tomorrow. I hope I'm productive today. I've actually been trying to plan. If I do the titles today, then I can do my laundry

In [12]:
BATCH_SIZE = 4
bucket_train_dataloader = DataLoader(train_data,
  batch_sampler=OpenDeidBatchSampler(train_data, BATCH_SIZE),
  collate_fn=lambda batch: collate_batch_with_prompt_template(batch, tokenizer),
  pin_memory=True)

## Training

In [13]:
import torch
torch.cuda.empty_cache()

In [15]:
from transformers import get_scheduler
from tqdm import tqdm
import torch
import os


EPOCHS = 8    #訓練次數5
GRADIENT_ACCUMULATION_STEPS = 4 #3
LEARNING_RATE = 5e-4
MAX_GRAD_NORM = 1.0
EARLY_STOP_PATIENCE = 6
output_dir = "/kaggle/working/qlora-deepseek"
os.makedirs(output_dir, exist_ok=True)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
model.train()
optimizer = torch.optim.AdamW(model.parameters(), lr=LEARNING_RATE)
total_steps = len(bucket_train_dataloader) * EPOCHS // GRADIENT_ACCUMULATION_STEPS
lr_scheduler = get_scheduler(
    "cosine",
    optimizer=optimizer,
    num_warmup_steps=200,
    num_training_steps=total_steps,
)
best_loss = float("inf")
early_stop_counter = 0

for epoch in range(EPOCHS):
    total_loss = 0.0
    model.train()

    progress_bar = tqdm(enumerate(bucket_train_dataloader), total=len(bucket_train_dataloader), desc=f"Epoch {epoch+1}")

    for step, (seqs, labels, masks) in progress_bar:
        input_ids = seqs.to(device)
        labels = labels.to(device)
        attention_mask = masks.to(device)

        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss / GRADIENT_ACCUMULATION_STEPS
        loss.backward()
        total_loss += loss.item()

        if (step + 1) % GRADIENT_ACCUMULATION_STEPS == 0 or (step + 1) == len(bucket_train_dataloader):
            torch.nn.utils.clip_grad_norm_(model.parameters(), MAX_GRAD_NORM)
            optimizer.step()
            lr_scheduler.step()
            model.zero_grad()

        progress_bar.set_postfix(
            loss=loss.item() * GRADIENT_ACCUMULATION_STEPS,
            lr=lr_scheduler.get_last_lr()[0]
        )

    avg_loss = total_loss / len(bucket_train_dataloader)
    print(f"\nEpoch {epoch+1} average loss: {avg_loss:.4f}")

    # ==== save best model ====
    if avg_loss < best_loss:
        best_loss = avg_loss
        early_stop_counter = 0
        best_path = os.path.join(output_dir, "best_adapter")
        model.save_pretrained(best_path)
        tokenizer.save_pretrained(best_path)
        print(f"Best model saved at {best_path} with loss {avg_loss:.4f}")
    else:
        early_stop_counter += 1
        print(f"No improvement. Early stop patience: {early_stop_counter}/{EARLY_STOP_PATIENCE}")

    # ==== Early Stopping ====
    if early_stop_counter >= EARLY_STOP_PATIENCE:
        print(f"Early stopping triggered at epoch {epoch+1}. Best loss: {best_loss:.4f}")
        break


model.save_pretrained(output_dir)
tokenizer.save_pretrained(output_dir)

Epoch 1:   0%|          | 0/2721 [00:00<?, ?it/s]`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.
/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
Epoch 1:  14%|█▍        | 388/2721 [16:24<1:38:41,  2.54s/it, loss=2.14, lr=0.000242]


KeyboardInterrupt: 

In [14]:
# 以下是你的訓練迴圈前的大段程式碼，需要進行大幅修改

from transformers import get_scheduler
from tqdm import tqdm
import torch
import os
from peft import PeftModel # 確保導入 PeftModel，它會用於從預訓練模型載入適配器

EPOCHS = 15     #訓練次數5
GRADIENT_ACCUMULATION_STEPS = 6 #3
LEARNING_RATE = 5e-4
MAX_GRAD_NORM = 1.0
EARLY_STOP_PATIENCE = 10
output_dir = "/kaggle/working/qlora-deepseek"
os.makedirs(output_dir, exist_ok=True)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# model 不再在這裡直接 to(device)，它會被包裝進 PeftModel 後再處理
# model.train() # 這行應該在所有模型準備好後再調用

best_loss = float("inf")
early_stop_counter = 0
start_epoch = 0 # 新增：記錄從哪個 epoch 開始訓練

# **請務必檢查你的 Kaggle Dataset，確認這個路徑是否正確**
# 載入 checkpoint.pth 的路徑應該只包含訓練進度信息，而不包含 LoRA 權重
# LoRA 權重應該單獨保存為 adapter_model.bin 和 adapter_config.json
# 假設你上次保存的 "best_adapter" 資料夾現在在你的輸入數據集中，
# 且路徑為 /kaggle/input/models/qlora-deepseek/best_adapter
adapter_load_path = "/kaggle/input/models/qlora-deepseek"
# 載入 checkpoint.pth (包含 optimizer, lr_scheduler 狀態) 的路徑
checkpoint_load_path = "/kaggle/input/models/qlora-deepseek/checkpoint.pth/checkpoint.pth" # 根據你上次錯誤，這個路徑可能是正確的

# **首先，嘗試從預訓練的適配器載入，這會直接創建一個 PeftModel 實例**
if os.path.exists(adapter_load_path) and os.path.isdir(adapter_load_path):
    print(f"載入 LoRA 適配器: {adapter_load_path}")
    model = PeftModel.from_pretrained(base_model, adapter_load_path)
    print("LoRA 適配器載入成功。")

    # 如果有檢查點，嘗試載入訓練進度（優化器、排程器狀態）
    if os.path.exists(checkpoint_load_path):
        print(f"載入檢查點: {checkpoint_load_path}")
        checkpoint = torch.load(checkpoint_load_path, map_location=device)

        # 重新初始化優化器和學習率排程器，然後載入其狀態
        optimizer = torch.optim.AdamW(model.parameters(), lr=LEARNING_RATE)
        optimizer.load_state_dict(checkpoint['optimizer_state_dict'])

        # 重新計算 total_steps_for_scheduler，因為我們可能只訓練剩餘的 epoch
        total_steps_remaining = len(bucket_train_dataloader) * (EPOCHS - (checkpoint['epoch'] + 1)) // GRADIENT_ACCUMULATION_STEPS
        lr_scheduler = get_scheduler(
            "cosine",
            optimizer=optimizer,
            num_warmup_steps=200, # 假設 warmup 步數在恢復訓練時保持不變
            num_training_steps=max(1, total_steps_remaining), # 確保至少為1，避免0除錯誤
        )
        lr_scheduler.load_state_dict(checkpoint['lr_scheduler_state_dict'])

        start_epoch = checkpoint['epoch'] + 1
        best_loss = checkpoint['best_loss']
        early_stop_counter = checkpoint['early_stop_counter']

        print(f"從 Epoch {start_epoch} 開始恢復訓練，最佳損失: {best_loss:.4f}，早期停止計數: {early_stop_counter}")
    else:
        # 如果有 LoRA 適配器但沒有檢查點，則從頭開始訓練（但LoRA已載入）
        print("未找到檢查點，從載入的 LoRA 適配器狀態開始訓練。")
        model = get_peft_model(base_model, lora_config) # 重新應用 LoRA 配置
        optimizer = torch.optim.AdamW(model.parameters(), lr=LEARNING_RATE)
        total_steps = len(bucket_train_dataloader) * EPOCHS // GRADIENT_ACCUMULATION_STEPS
        lr_scheduler = get_scheduler(
            "cosine",
            optimizer=optimizer,
            num_warmup_steps=200,
            num_training_steps=total_steps,
        )
else:
    # 如果沒有找到 LoRA 適配器資料夾，從頭開始訓練 (包括應用 LoRA 配置)
    print("未找到 LoRA 適配器，從頭開始訓練。")
    model = get_peft_model(base_model, lora_config) # 應用 LoRA 配置
    optimizer = torch.optim.AdamW(model.parameters(), lr=LEARNING_RATE)
    total_steps = len(bucket_train_dataloader) * EPOCHS // GRADIENT_ACCUMULATION_STEPS
    lr_scheduler = get_scheduler(
        "cosine",
        optimizer=optimizer,
        num_warmup_steps=200,
        num_training_steps=total_steps,
    )

model.to(device) # 將模型移到設備上
model.train() # 確保模型處於訓練模式

# ==== 訓練迴圈中的保存檢查點 ====
# 這裡的 checkpoint_path 指的是每次保存訓練進度的目標路徑。
checkpoint_path = os.path.join(output_dir, "checkpoint.pth") # 仍然是這個檔案路徑

# ==== 訓練迴圈 ====
model.train() # 確保模型處於訓練模式

for epoch in range(start_epoch, EPOCHS): # 修改迴圈起始點
    total_loss = 0.0
    model.train()

    progress_bar = tqdm(enumerate(bucket_train_dataloader), total=len(bucket_train_dataloader), desc=f"Epoch {epoch+1}")

    for step, (seqs, labels, masks) in progress_bar:
        input_ids = seqs.to(device)
        labels = labels.to(device)
        attention_mask = masks.to(device)

        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss / GRADIENT_ACCUMULATION_STEPS
        loss.backward()
        total_loss += loss.item()

        if (step + 1) % GRADIENT_ACCUMULATION_STEPS == 0 or (step + 1) == len(bucket_train_dataloader):
            torch.nn.utils.clip_grad_norm_(model.parameters(), MAX_GRAD_NORM)
            optimizer.step()
            lr_scheduler.step()
            model.zero_grad()

        progress_bar.set_postfix(
            loss=loss.item() * GRADIENT_ACCUMULATION_STEPS,
            lr=lr_scheduler.get_last_lr()[0]
        )

    avg_loss = total_loss / len(bucket_train_dataloader)
    print(f"\nEpoch {epoch+1} average loss: {avg_loss:.4f}")

    # ==== 保存最佳模型 ====
    # 這部分邏輯與之前相同，但可以考慮額外保存一個通用的最新檢查點
    if avg_loss < best_loss:
        best_loss = avg_loss
        early_stop_counter = 0
        best_path = os.path.join(output_dir, "best_adapter")
        model.save_pretrained(best_path) # PEFT 模型保存方法
        tokenizer.save_pretrained(best_path)
        print(f"最佳模型保存於 {best_path}，損失 {avg_loss:.4f}")
    else:
        early_stop_counter += 1
        best_path = output_dir
        model.save_pretrained(best_path) # PEFT 模型保存方法
        tokenizer.save_pretrained(best_path)
        print(f"無改善。早期停止耐心: {early_stop_counter}/{EARLY_STOP_PATIENCE}")

    # ==== 每次 epoch 結束時保存檢查點 (可選，但建議) ====
    # 這確保即使沒有達到最佳，你也能從最近的點恢復
    torch.save({
        'epoch': epoch,
        # 'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'lr_scheduler_state_dict': lr_scheduler.state_dict(),
        'best_loss': best_loss,
        'early_stop_counter': early_stop_counter,
    }, checkpoint_path)
    print(f"檢查點已保存至 {checkpoint_path}")

    # ==== 早期停止 ====
    if early_stop_counter >= EARLY_STOP_PATIENCE:
        print(f"在 Epoch {epoch+1} 觸發早期停止。最佳損失: {best_loss:.4f}")
        break

# 最後保存最終模型
model.save_pretrained(output_dir)
tokenizer.save_pretrained(output_dir)

載入 LoRA 適配器: /kaggle/input/models/qlora-deepseek
LoRA 適配器載入成功。
載入檢查點: /kaggle/input/models/qlora-deepseek/checkpoint.pth/checkpoint.pth
從 Epoch 6 開始恢復訓練，最佳損失: 0.0426，早期停止計數: 1


Epoch 7:   0%|          | 0/2913 [00:00<?, ?it/s]`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.
/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
Epoch 7: 100%|██████████| 2913/2913 [2:02:52<00:00,  2.53s/it, loss=0.929, lr=3.56e-5]  



Epoch 7 average loss: 0.1632


/usr/local/lib/python3.11/dist-packages/peft/utils/save_and_load.py:260: UserWarning: Setting `save_embedding_layers` to `True` as the embedding layer has been resized during finetuning.
  warnings.warn(


無改善。早期停止耐心: 2/10
檢查點已保存至 /kaggle/working/qlora-deepseek/checkpoint.pth


...


Epoch 10 average loss: 0.1631
無改善。早期停止耐心: 5/10
檢查點已保存至 /kaggle/working/qlora-deepseek/checkpoint.pth


Epoch 11: 100%|██████████| 2913/2913 [2:03:40<00:00,  2.55s/it, loss=1.28, lr=0.000355]   



Epoch 11 average loss: 0.1631
無改善。早期停止耐心: 6/10
檢查點已保存至 /kaggle/working/qlora-deepseek/checkpoint.pth


Epoch 12:   2%|▏         | 49/2913 [02:28<2:25:04,  3.04s/it, loss=0.579, lr=0.000357]


KeyboardInterrupt: 

## Load finetune model

In [4]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig,AutoConfig
from peft import PeftModel

bos = '<|endoftext|>'
eos = '<|END|>'
pad = '<|pad|>'
sep = '\n\n####\n\n'
base_model_name = "deepseek-ai/deepseek-llm-7b-base"
adapter_path = "/kaggle/working/qlora-deepseek/best_adapter"           # 訓練完要直接進行推論，用這行
# adapter_path = "/kaggle/input/models/qlora-deepseek"                   # 直接進行推論，用這行
tokenizer = AutoTokenizer.from_pretrained(adapter_path, trust_remote_code=True)
tokenizer.padding_side = 'left'
config = AutoConfig.from_pretrained(
    base_model_name,
    bos_token_id=tokenizer.bos_token_id,
    eos_token_id=tokenizer.eos_token_id,
    pad_token_id=tokenizer.pad_token_id,
    sep_token_id=tokenizer.sep_token_id,
    output_hidden_states=False
)

bnb_config = BitsAndBytesConfig(load_in_4bit=True, bnb_4bit_quant_type="nf4", bnb_4bit_use_double_quant=True)
base_model = AutoModelForCausalLM.from_pretrained(base_model_name,config=config, quantization_config=bnb_config, trust_remote_code=True)
base_model.resize_token_embeddings(len(tokenizer))

model = PeftModel.from_pretrained(base_model, adapter_path)
model.eval().cuda()

config.json:   0%|          | 0.00/584 [00:00<?, ?B/s]

`low_cpu_mem_usage` was None, now default to True since model is quantized.


pytorch_model.bin.index.json:   0%|          | 0.00/22.5k [00:00<?, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

pytorch_model-00002-of-00002.bin:   0%|          | 0.00/3.85G [00:00<?, ?B/s]

pytorch_model-00001-of-00002.bin:   0%|          | 0.00/9.97G [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.6k [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/121 [00:00<?, ?B/s]

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): LlamaForCausalLM(
      (model): LlamaModel(
        (embed_tokens): Embedding(100019, 4096, padding_idx=100017)
        (layers): ModuleList(
          (0-29): 30 x LlamaDecoderLayer(
            (self_attn): LlamaAttention(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=4096, out_features=4096, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4096, out_features=8, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=8, out_features=4096, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
         

## Output result

In [5]:
torch.cuda.empty_cache()

In [6]:
task2_valid_data = '/kaggle/input/audio-dataset/submission/task1_answer.txt'
valid_data = load_dataset("csv", data_files=task2_valid_data, delimiter='\t',
  features = Features({'fid': Value('string'),'content': Value('string')}),
  column_names=['fid', 'content'])
valid_list = list(valid_data['train'])
valid_data

Generating train split: 0 examples [00:00, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['fid', 'content'],
        num_rows: 710
    })
})

In [7]:
with open(r'/kaggle/working/task1_answer_timestamps.json', 'r', encoding='utf-8') as file:
  audio_timestamps = json.load(file)

In [1]:
# 沒有加入fuzzy match

import re
import json
from collections import defaultdict
import torch
from tqdm import tqdm  # 用來顯示進度條

# ========== 常用函數 ==========
def normalize_word(word):
    return re.sub(r'[^\w\s]', '', word.lower().strip())

def normalize_phi_token(token, phi_type=None):
    token = token.strip().lower()

    if phi_type in {'DATE', 'TIME', 'IDNUM', 'MEDICALRECORD', 'ACCOUNT'}:
        return re.sub(r'[^a-zA-Z0-9]', '', token)  # 保留字母和數字，去除特殊符號（. - 等）
    else:
        return re.sub(r'[^\w\s]', '', token)  # 原始處理

# ========== 標註類別 ==========
train_phi_category = [
    'PATIENT', 'DOCTOR', 'USERNAME', 'FAMILYNAME', 'PROFESSION',
    'ROOM', 'DEPARTMENT', 'HOSPITAL', 'ORGANIZATION', 'STREET', 'CITY',
    'DISTRICT', 'COUNTY', 'STATE', 'COUNTRY', 'ZIP', 'LOCATION-OTHER',
    'AGE', 'DATE', 'TIME', 'DURATION', 'SET',
    'PHONE', 'FAX', 'EMAIL', 'URL', 'IPADDR',
    'SSN', 'MEDICALRECORD', 'HEALTHPLAN', 'ACCOUNT',
    'LICENSE', 'VEHICLE', 'DEVICE', 'BIOID', 'IDNUM',
    'OTHER'
]

# ========== 匹配 timestamp 時間區間 ==========
def get_anno_format(infos, segments):
    anno_list = []
    phi_dict = defaultdict(list)

    for line in infos.split("\n"):
        if ":" not in line:
            continue
        key, value = line.split(":", 1)
        key = key.strip()
        value = value.strip()
        if key in train_phi_category and value:
            phi_dict[key].append(value)

    used_indices = set()

    for phi_key, phi_values in phi_dict.items():
        for phi_value in phi_values:
            norm_phi = normalize_phi_token(phi_value, phi_key)

            matched = False
            for i in range(len(segments)):
                MAX_WINDOW = 8 # 可再根據效能調整
                for j in range(i+1, min(i+MAX_WINDOW, len(segments)+1)):
                    if any(k in used_indices for k in range(i, j)):
                        continue
                    window_words = segments[i:j]
                    window_text = normalize_phi_token(" ".join([normalize_word(w['word']) for w in window_words]))

                    if window_text == norm_phi:
                        anno_list.append({
                            "phi": phi_key,
                            "st_time": segments[i]['start'],
                            "ed_time": segments[j-1]['end'],
                            "entity": phi_value
                        })
                        print(f'✅ Matched: phi="{phi_value}" → "{window_text}"')
                        used_indices.update(range(i, j))
                        matched = True
                        break
                if matched:
                    break
            if not matched:
                print(f'❌ No match: phi="{phi_value}", norm="{norm_phi}"')

    return anno_list

# ========== 預測函數 ==========
def aicup_predict(model, tokenizer, _input, audio_timestamps, template="<|endoftext|> __CONTENT__\n\n####\n\n"):
    outputs = []
    model.eval()
    device = model.device

    sep = tokenizer.sep_token or "\n####\n"
    eos = tokenizer.eos_token or "<|END|>"
    pad = tokenizer.pad_token or "<|pad|>"

    for item in tqdm(_input):
        seed = template.replace("__CONTENT__", item['content'])
        input_ids = tokenizer(seed, return_tensors='pt').to(device)

        with torch.no_grad():
            output_tokens = model.generate(
                **input_ids,
                max_new_tokens=128,
                do_sample=False,
                temperature=1.0,
                top_p=None,
                pad_token_id=tokenizer.pad_token_id,
                eos_token_id=tokenizer.eos_token_id
            )

        pred = tokenizer.decode(output_tokens[0], skip_special_tokens=False)

        # 擷取標註區塊
        if sep in pred:
            phi_infos = pred.split(sep, 1)[1]
        else:
            print(f"⚠️ Warning: sep_token not found in prediction.")
            continue

        phi_infos = phi_infos.replace(pad, "").replace(eos, "").replace("<|endoftext|>", "").strip()
        phi_infos = phi_infos.replace("\\n", "\n")

        fid = item['fid']
        segments = audio_timestamps[fid]["segments"]
        annotations = get_anno_format(phi_infos, segments)

        for annotation in annotations:
            outputs.append(f'{fid}\t{annotation["phi"]}\t{annotation["st_time"]}\t{annotation["ed_time"]}\t{annotation["entity"]}')

    return outputs

In [10]:
import re
import json
from collections import defaultdict
import torch
from tqdm import tqdm  # 用來顯示進度條
from difflib import SequenceMatcher

# ========== 常用函數 ==========
def normalize_word(word):
    return re.sub(r'[^\w\s]', '', word.lower().strip())

def normalize_phi_token(token, phi_type=None):
    token = token.strip().lower()
    if phi_type in {'DATE', 'TIME'}:
        return re.sub(r'[^a-zA-Z0-9:. ]', '', token)  # 保留空白和冒號句點
    elif phi_type in {'ID_NUMBER', 'MEDICAL_RECORD_NUMBER', 'ACCOUNT_NUMBER'}:
        return re.sub(r'[^a-zA-Z0-9]', '', token)
    else:
        return re.sub(r'[^\w\s]', '', token)

# ========== 標註類別 ==========
train_phi_category = [
    'PATIENT', 'DOCTOR', 'USERNAME','PERSONALNAME', 'FAMILYNAME', 'PROFESSION',
    'ROOM', 'DEPARTMENT', 'HOSPITAL', 'ORGANIZATION', 'STREET', 'CITY', 'STATE',
    'COUNTRY', 'COUNTY', 'ZIP', 'LOCATION-OTHER','DISTRICT',
    'AGE', 'DATE', 'TIME', 'DURATION', 'SET',
    'PHONE', 'FAX', 'EMAIL', 'URL', 'IPADDRESS','OTHER',
    'SOCIAL_SECURITY_NUMBER','MEDICAL_RECORD_NUMBER', 'HEALTH_PLAN_NUMBER', 'ACCOUNT_NUMBER',
    'LICENSE_NUMBER', 'VEHICLE_ID', 'DEVICE_ID', 'BIOMETRIC_ID', 'ID_NUMBER'
]

# ========== 匹配 timestamp 時間區間 ==========
def get_anno_format(infos, segments, fuzzy_threshold=0.9):
    anno_list = []
    phi_dict = defaultdict(list)

    for line in infos.split("\n"):
        if ":" not in line:
            continue
        key, value = line.split(":", 1)
        key = key.strip()
        value = value.strip()
        if key in train_phi_category and value:
            phi_dict[key].append(value)

    used_indices = set()

    for phi_key, phi_values in phi_dict.items():
        for phi_value in phi_values:
            # norm_phi = normalize_phi_token(phi_value)
            fuzzy_threshold_current = fuzzy_threshold
            if phi_key in {"TIME", "HOSPITAL", "CITY", "STATE", "STREET", "ZIP", "LOCATION-OTHER"}:
                fuzzy_threshold_current = 0.75# 可考慮再降低

            norm_phi = normalize_phi_token(phi_value, phi_key)
            matched = False
            best_match = None
            best_score = 0.0

            for i in range(len(segments)):
                MAX_WINDOW = 5
                for j in range(i+1, min(i+MAX_WINDOW, len(segments)+1)):
                    if any(k in used_indices for k in range(i, j)):
                        continue
                    window_words = segments[i:j]
                    window_text = normalize_phi_token(" ".join([normalize_word(w['word']) for w in window_words]))

                    # 1. exact match
                    if window_text == norm_phi:
                        anno_list.append({
                            "phi": phi_key,
                            "st_time": segments[i]['start'],
                            "ed_time": segments[j-1]['end'],
                            "entity": phi_value
                        })
                        print(f'✅ Matched: phi="{phi_value}" → "{window_text}" (exact)')
                        used_indices.update(range(i, j))
                        matched = True
                        break
                    else:
                        # 2. fuzzy match
                        score = SequenceMatcher(None, window_text, norm_phi).ratio()
                        # print(f"🧪 Trying fuzzy: '{norm_phi}' vs '{window_text}' → score={score:.3f}")
                        if score > best_score:
                            best_score = score
                            best_match = (i, j, window_text)

                if matched:
                    break

            if not matched and best_score >= fuzzy_threshold_current:
                i, j, window_text = best_match
                anno_list.append({
                    "phi": phi_key,
                    "st_time": segments[i]['start'],
                    "ed_time": segments[j-1]['end'],
                    "entity": phi_value
                })
                used_indices.update(range(i, j))
                print(f'🟡 Fuzzy Matched: phi="{phi_value}" ~ "{window_text}" (score={best_score:.2f}) [vs norm="{norm_phi}"]')
            elif not matched:
                # pass 
                print(f'❌ No match: phi="{phi_value}", norm="{norm_phi}"')

    return anno_list

# ========== 預測函數 ==========
def aicup_predict(model, tokenizer, _input, audio_timestamps, template="<|endoftext|> __CONTENT__\n\n####\n\n"):
    outputs = []
    model.eval()
    device = model.device

    sep = tokenizer.sep_token or "\n####\n"
    eos = tokenizer.eos_token or "<|END|>"
    pad = tokenizer.pad_token or "<|pad|>"

    for item in tqdm(_input):

        content = item.get('content')
        if content is None:
            content = "" # 將 None 替換為空字串
        
        seed = template.replace("__CONTENT__", content)
        
        input_ids = tokenizer(seed, return_tensors='pt').to(device)

        with torch.no_grad():
            output_tokens = model.generate(
                **input_ids,
                max_new_tokens=128,
                do_sample=False,
                temperature=1.0,
                top_p=None,
                pad_token_id=tokenizer.pad_token_id,
                eos_token_id=tokenizer.eos_token_id
            )

        pred = tokenizer.decode(output_tokens[0], skip_special_tokens=False)

        # 擷取標註區塊
        if sep in pred:
            phi_infos = pred.split(sep, 1)[1]
        else:
            print(f"⚠️ Warning: sep_token not found in prediction.")
            continue

        phi_infos = phi_infos.replace(pad, "").replace(eos, "").replace("<|endoftext|>", "").strip()
        phi_infos = phi_infos.replace("\\n", "\n")

        fid = item['fid']
        segments = audio_timestamps[fid]["segments"]
        annotations = get_anno_format(phi_infos, segments)

        for annotation in annotations:
            outputs.append(f'{fid}\t{annotation["phi"]}\t{annotation["st_time"]}\t{annotation["ed_time"]}\t{annotation["entity"]}')

    return outputs

In [11]:
import torch
from tqdm import tqdm

BATCH_SIZE = 30
with open("/kaggle/working/task2_answer.txt",'w',encoding='utf8') as f:
  for i in tqdm(range(0, len(valid_list), BATCH_SIZE)):
    with torch.no_grad():
      data = valid_list[i:i+BATCH_SIZE]
      outputs = aicup_predict(model, tokenizer, data, audio_timestamps)
      for o in outputs:
        f.write(o)
        f.write('\n')

  3%|▎         | 2/60 [00:02<01:16,  1.33s/it]

✅ Matched: phi="75J64061" → "75j64061" (exact)



  7%|▋         | 4/60 [00:24<07:01,  7.53s/it]

🟡 Fuzzy Matched: phi="16Q507751V" ~ "16q500775v" (score=0.90) [vs norm="16q507751v"]
✅ Matched: phi="Vita Lawson" → "vita lawson" (exact)
✅ Matched: phi="Vita" → "vita" (exact)
✅ Matched: phi="W. Goodenow Street" → "w goodenow street" (exact)
✅ Matched: phi="Gobachu" → "gobachu" (exact)
✅ Matched: phi="Tasmania" → "tasmania" (exact)
✅ Matched: phi="3995" → "3995" (exact)
✅ Matched: phi="St. Vincent's Hospital" → "st vincents hospital" (exact)
✅ Matched: phi="Dollinghurst" → "dollinghurst" (exact)



  8%|▊         | 5/60 [00:29<06:02,  6.58s/it]

...

✅ Matched: phi="165443" → "165443" (exact)
✅ Matched: phi="160443405T" → "160443405t" (exact)
✅ Matched: phi="July 5, 2033" → "july 5 2033" (exact)
✅ Matched: phi="October 22, 2063" → "october 22 2063" (exact)
✅ Matched: phi="Wales" → "wales" (exact)
✅ Matched: phi="Wombat" → "wombat" (exact)
✅ Matched: phi="New South Wales" → "new south wales" (exact)
✅ Matched: phi="5141" → "5141" (exact)
✅ Matched: phi="Hereditary Department" → "hereditary department" (exact)



 98%|█████████▊| 49/50 [04:40<00:07,  7.20s/it]

✅ Matched: phi="30, 2062" → "30 2062" (exact)



100%|██████████| 12/12 [1:07:15<00:00, 336.25s/it]

❌ No match: phi="000000000", norm="000000000"
